In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
import keras
from keras import layers

2025-06-23 17:38:53.003518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750700333.219644      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750700333.283230      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
class complexConv2D(layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='same'):
        super(complexConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding

        self.real_conv2D = layers.Conv2D(filters, kernel_size, strides = strides, padding = padding)
        self.complex_conv2D = layers.Conv2d(filters, kernel_size, strides = strides, padding = padding)

    def call(self, input_stft):
        real_stft, img_stft = tf.split(input_stft, axis=-1)

        real_stft_real = self.real_conv2D(real_stft)
        real_stft_img = self.complex_conv2D(real_stft)

        img_stft_real = self.real_conv2D(img_stft)
        img_stft_img = self.complex_conv2D(img_stft)

        output_real = real_stft_real - img_stft_img
        output_img = real_stft_img + img_stft_real

        return tf.stack([output_real, output_img], axis=-1)
        

In [4]:
class complexBN_PReLu(layers.Layer):
    def __init__(self):
        super(complexBN_PReLu, self).__init__()
        self.real_bn = layers.BatchNormalization()
        self.img_bn = layers.BatchNormalization()
        self.real_prelu = layers.prelu()
        self.img_prelu = layers.prelu()


    def call(self, inputs):
        real, img = tf.split(inputs, 2, axis=-1)

        real = self.real_bn(real)
        img = self.img_bn(img)

        real = self.real_prelu(real)
        img = self.img_prelu(img)

        return tf.stack([real, img], axis=-1)

In [5]:
class complexEncodeBlock(layers.Layer):
    def __init__(self, filters, kernel_size=(3,3), strides=(1,1)):
        super(complexEncodeBlock, self).__init__()
        self.conv = complexConv2D(filters, kernel_size, strides=strides)
        self.bn_prelu = complexBN_PReLu()

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn_prelu(x)
        return x